In [2]:
import random
n=10
V = [(random.uniform(5,75),random.uniform(5,55)) for i in range(n)]
import matplotlib.pyplot as plt
alpha=2

In [3]:
def t_cost(s,t): return ((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5)
def d_cost(s,t): return (1/alpha)*(((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5))

In [4]:
import itertools
def two_opt(ps):
    n = len(ps)
    res = list(range(n))
    update = True
    while update:
        update = False
        for (i,j) in itertools.combinations(range(n),2):
            if ((t_cost(ps[res[i]],ps[res[i+1]])+t_cost(ps[res[j]],ps[res[(j+1)%n]]))>
                (t_cost(ps[res[i]],ps[res[j]])+t_cost(ps[res[i+1]],ps[res[(j+1)%n]]))):
                res[i+1:j+1] = res[j:i:-1]
                update = True
                
    return res

In [5]:
import heapq
def mst(ps):
    n = len(ps)
    #prim
    mst = {v:[] for v in range(n)} # MST の隣接リスト
    X = set()
    heap = []
    for v in range(1,n):
        heapq.heappush(heap,(t_cost(ps[0],ps[v]),0,v))
    while len(heap)>0:
        (d,u,v)=heapq.heappop(heap)
        if v not in X:
            X.add(v)
            mst[u].append(v)
            mst[v].append(u)
            for w in range(n):
                if w not in X: heapq.heappush(heap,(t_cost(ps[v],ps[w]),v,w))
    #dfs
    res = []
    stack = [0]
    visited = [False]*n
    while len(stack)>0:
        v = stack.pop()
        if visited[v]: continue
        visited[v]=True
        res.append(v)
        for u in mst[v]: stack.append(u)
    return res 

In [6]:
mst(V)

[0, 8, 2, 5, 4, 6, 3, 1, 9, 7]

In [9]:
def dp(ps):
    length = {} # length[(u,S)]: u を始点とし S の点すべてを回る最小経路長、ディクショナリ
    route = {} # route[(u,S)]: 最小経路長を達成するためのルート、ディクショナリ
    v = ps[0]
    n = len(ps)
    for i in range(1,n+1):
        for a in itertools.combinations(range(n),i):
            S = frozenset(a)
            for j in S:
                u = ps[j]
                if i==1:
                    length[(j,S)] = t_cost(v,u)
                    route[(j,S)] = [j]
                else:
                    Sj = S-set([j]) # S から j を除いたもの
                    k=min(Sj,key=lambda k: length[(k,Sj)]+t_cost(ps[k],u))#Sjの中でlength[(k,Sj)]+dist(ps[k],u)が最小のものを返している
                    length[(j,S)] = length[(k,Sj)]+t_cost(ps[k],u)
                    route[(j,S)] = route[(k,Sj)]+[j]
    return route[(0,frozenset(range(n)))] 

In [1]:
def t_pathcost(a,b,route):
    pathcost=0
    if a<b:
        for i in range(a,b):
            pathcost=pathcost+t_cost(V[route[i]],V[route[(i+1)%len(route)]])
    if a>b:
        for i in range(a,len(route)):
            pathcost=pathcost+t_cost(V[route[i]],V[route[(i+1)%len(route)]])
        for i in range(b):
            pathcost=pathcost+t_cost(V[route[i]],V[route[(i+1)%len(route)]])
    if a==b:
        pathcost=0
            
    return pathcost

In [11]:
import itertools
def exact_core(tsp,depot):
    T={}
    M={}
    D={}
    Dres=0
    Mdrone={}
    route={}
    n=len(tsp)
    for (start,drone,end) in itertools.combinations(range(n),3):
        drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
        truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[drone+1]])+t_pathcost(drone+1,end,tsp)
        T[start,drone,end]=max(drone_cost,truck_cost)
    for (end,start,drone) in itertools.combinations(range(n),3):
        drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
        truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[(drone+1)%n]])+t_pathcost((drone+1)%n,end,tsp)
        T[start,drone,end]=max(drone_cost,truck_cost)
    for (drone,end,start) in itertools.combinations(range(n),3):
        drone_cost=d_cost(V[tsp[start]],V[tsp[drone]])+d_cost(V[tsp[drone]],V[tsp[end]])
        truck_cost=t_pathcost(start,drone-1,tsp)+t_cost(V[tsp[drone-1]],V[tsp[(drone+1)%n]])+t_pathcost((drone+1)%n,end,tsp)
        T[start,drone,end]=max(drone_cost,truck_cost)
        
    for (i,j) in itertools.permutations(range(n),2):
        M[i,j]=float('inf')
        if i<j:
            if i+1==j:
                M[i,j]=t_cost(V[tsp[i]],V[tsp[j]])
            for k in range(i+1,j):
                if M[i,j]>T[i,k,j]:
                    M[i,j]=T[i,k,j]
                    Mdrone[tsp[i],tsp[j]]=[tsp[k]]
        else:
            for k in range(i+1,n):
                if M[i,j]>T[i,k,j]:
                    M[i,j]=T[i,k,j]
                    Mdrone[tsp[i],tsp[j]]=[tsp[k]]
            for k in range(0,j):
                if M[i,j]>T[i,k,j]:
                    M[i,j]=T[i,k,j]
                    Mdrone[tsp[i],tsp[j]]=[tsp[k]]
            M[n-1,0]=t_cost(V[tsp[n-1]],V[tsp[0]])

    for i in range(tsp.index(depot)+1,tsp.index(depot)+n+1):
        D[depot,tsp[i%n]]=float('inf')
        Dres=float('inf')
        D[depot,depot]=0
        route[depot,depot]=[[depot]]
        for j in range(tsp.index(depot),i):
            if j%n!=i%n:
                d=M[j%n,i%n]+D[depot,tsp[j%n]]
                r=route[depot,tsp[j%n]]+[[tsp[j%n],tsp[i%n]]]
                if i==tsp.index(depot)+n:
                    if Dres>d:
                        Dres=d
                        route[depot,depot]=r

                else:
                    if D[depot,tsp[i%n]]>d:
                        D[depot,tsp[i%n]]=d
                        route[depot,tsp[i%n]]=r


    pathlen=len(route[depot,depot])
    path=[]
    for i in range(pathlen-1):
        path=path+route[depot,depot][i+1]
        if (route[depot,depot][i+1][0],route[depot,depot][i+1][1]) in Mdrone:
            path=path+[Mdrone[route[depot,depot][i+1][0],route[depot,depot][i+1][1]]]
        else:
            path=path+[[]]
    
        
    
    return Dres,path

In [12]:
import itertools
def exact_ph(alg,V,depot):
    tsp=alg(V)
    return exact_core(tsp,depot)

In [13]:
exact_ph(two_opt,V,4)

(104.51494635935934, [4, 7, [6], 7, 0, [8], 0, 4, [2]])